In [1]:
import nltk
import numpy as np
import pandas as pd
import string
import json
import os
import csv
from collections import Counter
from json.decoder import JSONDecodeError
import itertools as it


In [2]:
def getHashtagCounts(hastagList):
    allHashtags = list(map(lambda x:x.lower(),hastagList))
    return Counter(allHashtags)

def getCoOccurrence(df,numField,field):
    hashtagsCoOccurence = {}
    for index, row in df.loc[df[numField]>1].iterrows(): # pick tweets with more than one hastag
        for h1,h2 in list(it.combinations(row[field], 2)): # Find all combinations for the list of hashtag
            #frozenset makes sure that order of the keys don't matter
            if frozenset([h1,h2]) in hashtagsCoOccurence.keys() and len(frozenset([h1,h2]))>1: 
                hashtagsCoOccurence[frozenset([h1,h2])] +=1
            else:
                hashtagsCoOccurence[frozenset([h1,h2])] = 1
    return Counter(hashtagsCoOccurence)

def getTokenFreq(df, dataField, isList):
    allTokens = []
    if isList:
        for x in df[dataField]:#.apply(lambda x: x.split(",")):
            allTokens+=x
    else:
        for x in df[dataField].apply(lambda x: x.split(",")):
            allTokens+=x
    return getHashtagCounts(allTokens)

def getDataOnKeyword(df,field,keyword): # Make sure that field you are searching in is a list
    return df[df[field].apply(lambda x: True if keyword in x else False)]

In [4]:
allFiles = []
for file in os.listdir("./data"):
    if file.endswith(".json"):
        allFiles.append(file)

In [5]:
tweets = {"tweetId":[],"postedTime":[],"body":[],"hashtags":[]}
fileNo = 0
for fileName in allFiles:
    fileNo+=1
    print(fileNo)
    data = []
    with open('./data/'+fileName) as f:
        for line in f:
            try:
                data = json.loads(line)
            except JSONDecodeError:
                continue
            
            if "lang" not in data.keys():
                continue
            if data["lang"] == "en" and len(data["entities"]["hashtags"])>0:
                tweets["tweetId"].append(data["id_str"])
                tweets["postedTime"].append(data["created_at"])
                tweets["body"].append(data["text"])
                hashtags = []
                for h in data["entities"]["hashtags"]:
                    hashtags.append(h["text"])
                tweets["hashtags"].append(hashtags)
            
            
            #jsonData = json.loads(line)
            #data.append(json.loads(line))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162


In [28]:
#i=0
i+=1
while len(data[i]["entities"]["hashtags"]) == 0:
    i+=1

data[i]["entities"]["hashtags"]

[{'indices': [63, 76], 'text': 'MasterChefMx'}]

In [15]:
i = 0
with open('./data/'+allFiles[46]) as f:
    for line in f:
        i+=1
        try:
            data = json.loads(line)
        except JSONDecodeError:
            continue
        #ptn
        
        

In [6]:
tweets["numHashtags"] = []
for hashlist in tweets["hashtags"]:
    tweets["numHashtags"].append(len(hashlist))

In [7]:
tweetDf = pd.DataFrame(tweets)

In [100]:
tweetDf["hashtags"] = tweetDf["hashtags"].apply(lambda x: ",".join(x))

In [101]:
tweetDf.to_csv("./data/tweets25.csv",quoting=csv.QUOTE_NONNUMERIC, date_format='%Y-%m-%d %H:%M:%S', encoding='utf-8')

In [3]:
checkData = pd.read_csv("./data/tweets25.csv", encoding='utf-8',index_col=0)
checkData["hashtags"] = checkData["hashtags"].apply(lambda x: str(x).lower())
checkData["hashtags"] = checkData["hashtags"].apply(lambda x: x.split(","))
checkData["body"] = checkData["body"].apply(lambda x: x.lower())
checkData["postedTime"] = pd.to_datetime(checkData["postedTime"])

In [11]:
allHashtags = []
for x in checkData["hashtags"]:#.apply(lambda x: x.split(",")):
    allHashtags+=x


In [4]:
allHashtagsCtr = getTokenFreq(checkData,"hashtags",1)

In [19]:
myTemp.most_common(10)

[('btshot100', 3752),
 ('takeaknee', 3182),
 ('taketheknee', 2230),
 ('bts', 2090),
 ('nowplaying', 1890),
 ('teamladygaga', 1875),
 ('hot100', 1859),
 ('teenwolf', 1854),
 ('nfl', 1803),
 ('puertorico', 1699)]

In [12]:
allHashtagsCtr = getHashtagCounts(allHashtags)

In [5]:
allHashtagsCtr.most_common(100)

[('btshot100', 3752),
 ('takeaknee', 3182),
 ('taketheknee', 2230),
 ('bts', 2090),
 ('nowplaying', 1890),
 ('teamladygaga', 1875),
 ('hot100', 1859),
 ('teenwolf', 1854),
 ('nfl', 1803),
 ('puertorico', 1699),
 ('giveaway', 1698),
 ('mondaymotivation', 1659),
 ('grahamcassidy', 1537),
 ('starmagicballfanfave', 1322),
 ('job', 1319),
 ('방탄소년단', 1252),
 ('aldubgenuinefeelings', 1216),
 ('win', 1206),
 ('got7', 1200),
 ('exo', 1154),
 ('maga', 1055),
 ('boycottnfl', 1000),
 ('hiring', 1000),
 ('news', 976),
 ('trapadrive', 928),
 ('갓세븐', 911),
 ('raw', 894),
 ('np', 854),
 ('trecru', 825),
 ('7for7', 814),
 ('standforouranthem', 768),
 ('dalvsaz', 710),
 ('trump', 675),
 ('aldubloveyourown', 661),
 ('music', 632),
 ('bitcoin', 611),
 ('soundcloud', 609),
 ('lab17', 597),
 ('dna', 596),
 ('ai', 576),
 ('blockchain', 569),
 ('rt', 569),
 ('free', 561),
 ('jobs', 559),
 ('ad', 558),
 ('spyder', 551),
 ('youare', 541),
 ('marketing', 523),
 ('breaking', 514),
 ('dallascowboys', 514),
 ('quot

In [115]:
checkData["hashtags"] = checkData["hashtags"].apply(lambda x: x.split(","))

In [97]:
len(checkData)

204413

In [6]:
startData = checkData[checkData["hashtags"].apply(lambda x: True if "takeaknee" in x else False)]

In [4]:
startData = getDataOnKeyword(checkData,"hashtags","takeaknee")
allHashtagsCtr = getTokenFreq(startData,"hashtags",1)
hashtagCoOccur = getCoOccurrence(startData,"numHashtags","hashtags")

In [101]:
len(startData)

3174

In [5]:
allHashtagsCtr.most_common(10)

[('takeaknee', 3182),
 ('nfl', 143),
 ('taketheknee', 93),
 ('boycottnfl', 79),
 ('goodtrouble', 53),
 ('jerryjones', 41),
 ('mnf', 36),
 ('maga', 33),
 ('takeastand', 26),
 ('trump', 24)]

In [6]:
hashtagCoOccur.most_common(10)

[(frozenset({'nfl', 'takeaknee'}), 143),
 (frozenset({'takeaknee', 'taketheknee'}), 93),
 (frozenset({'boycottnfl', 'takeaknee'}), 79),
 (frozenset({'goodtrouble', 'takeaknee'}), 53),
 (frozenset({'jerryjones', 'takeaknee'}), 41),
 (frozenset({'mnf', 'takeaknee'}), 36),
 (frozenset({'maga', 'takeaknee'}), 33),
 (frozenset({'takeaknee', 'takeastand'}), 26),
 (frozenset({'takeaknee', 'trump'}), 24),
 (frozenset({'dallascowboys', 'takeaknee'}), 22)]

In [133]:
hashtagList = []
for x in startData["hashtags"]:
    hashtagList+=x
hashtagListCtr = getHashtagCounts(hashtagList)

In [134]:
hashtagListCtr.most_common(15)

[('takeaknee', 3182),
 ('nfl', 143),
 ('taketheknee', 93),
 ('boycottnfl', 79),
 ('goodtrouble', 53),
 ('jerryjones', 41),
 ('mnf', 36),
 ('maga', 33),
 ('takeastand', 26),
 ('trump', 24),
 ('dallascowboys', 22),
 ('mondaynightfootball', 21),
 ('blacklivesmatter', 18),
 ('takeakneenfl', 18),
 ('standforouranthem', 17)]

In [8]:
hashtagCoOccur = getCoOccurrence(startData,"numHashtags","hashtags")

In [9]:
hashtagCoOccur.most_common(10)

[(frozenset({'nfl', 'takeaknee'}), 143),
 (frozenset({'takeaknee', 'taketheknee'}), 93),
 (frozenset({'boycottnfl', 'takeaknee'}), 79),
 (frozenset({'goodtrouble', 'takeaknee'}), 53),
 (frozenset({'jerryjones', 'takeaknee'}), 41),
 (frozenset({'mnf', 'takeaknee'}), 36),
 (frozenset({'maga', 'takeaknee'}), 33),
 (frozenset({'takeaknee', 'takeastand'}), 26),
 (frozenset({'takeaknee', 'trump'}), 24),
 (frozenset({'dallascowboys', 'takeaknee'}), 22)]

In [25]:
exec(compile(open('repl.py', "rb").read(), 'repl.py', 'exec'))

Loading the model, this can take some time...


MemoryError: 

In [4]:
import gensim

C:\Program Files\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [26]:
model = gensim.models.KeyedVectors.load_word2vec_format('./glove.twitter.27B/w2v.twitter.27B.200d.txt', binary=False)  

In [49]:
model.most_similar([","])

[('.', 0.7419614791870117),
 (';', 0.6766710877418518),
 ('<repeat>', 0.6555314064025879),
 ('?', 0.6401088237762451),
 ('ok', 0.6091393828392029),
 ('"', 0.6055634021759033),
 ('<user>', 0.5923637747764587),
 ('rt', 0.5861671566963196),
 (':', 0.5844361782073975),
 ('&', 0.5822509527206421)]

In [5]:
import string
import re
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import sent_tokenize, word_tokenize
tknz = TweetTokenizer()
from nltk.corpus import stopwords
stop = ["rt"] #+ stopwords.words('english') + list(string.punctuation)
from gensim.models.doc2vec import (
    Doc2Vec,
    TaggedDocument,
)
# translator = str.maketrans('', '', string.punctuation.replace("#","").replace("@","").replace("'",""))
translator = str.maketrans('', '', "@#")


def cleanTweet(text):
    text = (re.sub(r"\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\W/]*))*", "", text)).lower() #removes urls
    #text = re.sub(r'[^\x00-\x7F]+',' ', text) # removes unicodes (emogis)
    text = re.sub(r'(\r)|(\n)','',text) # removes newline characters
    text = text.replace("rt ","")
    text= text.translate(translator) 
#     tokens = word_tokenize(text)
    tokens = tknz.tokenize(text)
#     temp=[]
#     for i in tokens:
#         if not i in stop:
#             temp.append(i)
    
    return tokens

# def labelizeDocs(dataList, label):
#     import gensim
#     LabeledSentence = gensim.models.doc2vec.LabeledSentence
#     labelized = []
#     for dataPoint in dataList:
#         labelized.append(LabeledSentence(dataPoint, [label]))
    
#     return labelized


def dataPrep(df,dataField):
    dataList = list(df[dataField].apply(lambda x: cleanTweet(x)))
    return dataList

def TaggDocs(dataList,label):
    newDataList = []
    for dataPoint in dataList:
        newDataList.append(TaggedDocument(dataPoint, tags=[label]))
    return newDataList

def getVecs(model, corpus, size):
    vecs = [np.array(model.docvecs[z.tags[0]]).reshape((1, size)) for z in corpus]
    return np.concatenate(vecs)


In [69]:
startData.columns

Index(['body', 'hashtags', 'numHashtags', 'postedTime', 'tweetId'], dtype='object')

In [130]:
len(vocabDict)

4979

In [11]:
vocabDict = dict(getHashtagCounts(np.sum(currentDataList)))

In [6]:
startData = getDataOnKeyword(checkData,"hashtags","takeaknee")
currentDataList = dataPrep(startData,"body")
currentData = TaggDocs(currentDataList,"takeaknee")

In [ ]:
modelD2V = Doc2Vec(vector_size = 150,min_count=1, workers=1)

modelD2V.build_vocab(currentData)
modelD2V.train(currentData, total_examples=modelD2V.corpus_count , epochs=100)

In [ ]:
print("-- Base model")
print("Vocabulary:", str(modelD2V.vocabulary))
print("Tags:", str(modelD2V.docvecs.doctags))


In [74]:
modelD2V.vocabulary.__dict__

{'cum_table': array([  43009431,   83489087,  116522242, ..., 2147285948, 2147384797,
        2147483647], dtype=uint32),
 'max_vocab_size': None,
 'min_count': 1,
 'null_word': 0,
 'raw_vocab': defaultdict(int, {}),
 'sample': 0.001,
 'sorted_vocab': True}

In [75]:
startData = getDataOnKeyword(checkData,"hashtags","nfl")
currentDataList = dataPrep(startData,"body")
currentData = TaggDocs(currentDataList,"nfl")

In [78]:
modelD2V.build_vocab(currentData, update=True)
modelD2V.train(currentData, total_examples=modelD2V.corpus_count , epochs=100)

In [37]:
modelD2V.wv.similarity('takeaknee',"")

TypeError: similarity() missing 1 required positional argument: 'w2'

In [40]:
modelD2V.infer_vector(currentData[1].words)

array([-0.14946002,  0.12858914, -0.11064957,  0.18100592,  0.06236762,
       -0.18181707, -0.0905913 ,  0.10728567, -0.05289251,  0.16354413,
       -0.05712029,  0.0676381 , -0.20695189, -0.15659273, -0.13450722,
       -0.15902491,  0.13131812, -0.10548742,  0.15689614, -0.11913933,
        0.08753449, -0.08534686, -0.0040849 ,  0.15699062,  0.06726166,
       -0.16104774,  0.20500545, -0.10828539, -0.06238169, -0.12093531,
       -0.14034757, -0.17053373,  0.04061782, -0.11544313, -0.15224779,
       -0.23189385, -0.2721566 ,  0.16871324,  0.25340274,  0.12745458,
       -0.03096354, -0.09017185,  0.14774773, -0.17303638, -0.12251407,
       -0.01375806, -0.06732075, -0.09686725,  0.1435098 ,  0.2123388 ,
       -0.09106575, -0.05397035,  0.09362867, -0.14578848,  0.3861563 ,
        0.0199358 ,  0.10730179, -0.06666312, -0.06678216, -0.09857281,
       -0.03787049,  0.21551861,  0.35128236,  0.08892806, -0.00980853,
        0.02956393, -0.0170132 , -0.01389343,  0.0733055 ,  0.16

In [55]:
vec1 = modelD2V.infer_vector(currentData[0].words)
vec2 = modelD2V.infer_vector(currentData[1].words)
#modelD2V.wv.wmdistance(currentData[0].words,currentData[1].words)

In [56]:
#modelD2V.docvecs.similar_by_vector(modelD2V.infer_vector(currentData[1].words),modelD2V.infer_vector(currentData[1].words))

In [24]:
currentData[1]

TaggedDocument(words=['1', ')', 'as', 'i', 'am', 'being', 'told', 'to', 'keep', 'quiet', ',', 'it', 'occurs', 'to', 'me', 'that', 'blessed', 'are', 'the', 'meek', 'who', "don't", 'stir', 'the', 'pot', '.', 'takeaknee'], tags=['takeaknee'])

In [57]:
# ranks = []
# second_ranks = []
# for doc_id in range(len(currentData)):
#     inferred_vector = modelD2V.infer_vector(currentData[doc_id].words)
#     sims = modelD2V.docvecs.most_similar([inferred_vector], topn=len(modelD2V.docvecs))
#     print(sims)
# #     rank = [docid for docid, sim in sims].index(doc_id)
# #     ranks.append(rank)
    
# #     second_ranks.append(sims[1])

In [58]:
# sims

In [ ]:
from gensim.models.word2vec import Word2Vec
modelW2V= Word2Vec()
modelW2V.build_vocab_from_freq(vocabDict)

In [115]:
currentDataList[0]

['brennanmgilmore',
 ':',
 'my',
 'grandpa',
 'is',
 'a',
 '97',
 'year-old',
 'wwii',
 'vet',
 '&',
 'missouri',
 'farmer',
 'who',
 'wanted',
 'to',
 'join',
 'w',
 '/',
 'those',
 'who',
 'takeaknee',
 ':',
 '"',
 'those',
 'kids',
 'have',
 '…']

In [124]:
modelD2V.infer_vector(['abcd'])

array([-2.05464102e-03,  2.27787020e-03, -2.32855906e-03, -9.30631883e-04,
       -2.75504310e-03, -6.96565490e-04,  2.41199392e-03,  1.51247112e-03,
        2.08194717e-03, -4.72134503e-04,  5.53112011e-04,  3.03332042e-03,
        3.21036601e-03,  6.18466293e-04,  5.23625233e-04,  2.32679420e-03,
       -2.17999122e-03, -3.16101545e-03, -1.63674331e-03, -3.06758890e-03,
        2.17901776e-03, -1.53846422e-03,  1.72459695e-03, -1.44430948e-03,
       -1.90755993e-03, -1.44501356e-03,  1.97073817e-03,  7.22338227e-05,
        1.50917750e-03,  8.89170740e-04,  7.97304208e-04, -4.75994195e-04,
        2.65102275e-03,  1.24810508e-03,  2.87641352e-03, -1.07126754e-04,
        2.03426834e-03, -1.19043025e-03, -1.19020150e-03, -1.69440324e-03,
       -2.89054005e-03, -2.79267016e-03, -2.74410169e-03, -1.15914889e-04,
        2.18764506e-03,  1.53955992e-03,  2.52419710e-03,  2.47155013e-03,
        3.31399613e-03,  2.46036984e-03, -1.91608886e-03, -9.61819082e-04,
       -5.16408822e-04,  

In [ ]:
model_dm = gensim.models.Doc2Vec(size=size, min_count=1, iter=100, alpha=0.025, min_alpha=0.0001)
model_dbow = gensim.models.Doc2Vec(size=size, min_count=1, dm=0, iter=100,alpha=0.025, min_alpha=0.0001)
    

model_dm.build_vocab(allData)
model_dbow.build_vocab(allData)   

    
model_dm.train(allData, total_examples=model_dm.corpus_count , epochs=100)   ######## allData not only training
model_dbow.train(allData, total_examples=model_dm.corpus_count , epochs=100)##### instead of epochs=model_dm.iter



train_vecs_dm = np.array(getVecs(model_dm, training, size))
train_vecs_dbow = np.array(getVecs(model_dbow, training, size))





In [ ]:
import logging

from gensim.models.doc2vec import (
    Doc2Vec,
    TaggedDocument,
)

logging.basicConfig(
    format='%(asctime)s : %(threadName)s : %(levelname)s : %(message)s',
    level=logging.DEBUG,
)


def to_str(d):
    return ", ".join(d.keys())


SENTS = [
    "anecdotal using a personal experience or an isolated example instead of a sound argument or compelling evidence",
    "plausible thinking that just because something is plausible means that it is true",
    "occam razor is used as a heuristic technique discovery tool to guide scientists in the development of theoretical models rather than as an arbiter between published models",
    "karl popper argues that a preference for simple theories need not appeal to practical or aesthetic considerations",
    "the successful prediction of a stock future price could yield significant profit",
]

SENTS = [s.split() for s in SENTS]



sentences_1 = [
        TaggedDocument(SENTS[0], tags=['SENT_0']),
        TaggedDocument(SENTS[1], tags=['SENT_0']),
        TaggedDocument(SENTS[2], tags=['SENT_1']),
    ]

sentences_2 = [
        TaggedDocument(SENTS[3], tags=['SENT_1']),
        TaggedDocument(SENTS[4], tags=['SENT_2']),
    ]

model = Doc2Vec(min_count=1, workers=1)

model.build_vocab(sentences_1)
model.train(sentences_1,total_examples=model.corpus_count,epochs=100)

print("-- Base model")
# print("Vocabulary:", to_str(model.vocab))
print("Tags:", to_str(model.docvecs.doctags))

model.build_vocab(sentences_2, update=True)
model.train(sentences_2,total_examples=model.corpus_count,epochs=100)

print("-- Updated model")
# print("Vocabulary:", to_str(model.vocab))
print("Tags:", to_str(model.docvecs.doctags))



2018-05-02 23:33:35,641 : MainThread : INFO : collecting all words and their counts
2018-05-02 23:33:35,643 : MainThread : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2018-05-02 23:33:35,644 : MainThread : INFO : collected 46 word types and 2 unique tags from a corpus of 3 examples and 57 words
2018-05-02 23:33:35,644 : MainThread : INFO : Loading a fresh vocabulary
2018-05-02 23:33:35,645 : MainThread : INFO : min_count=1 retains 46 unique words (100% of original 46, drops 0)
2018-05-02 23:33:35,646 : MainThread : INFO : min_count=1 leaves 57 word corpus (100% of original 57, drops 0)
2018-05-02 23:33:35,647 : MainThread : INFO : deleting the raw counts dictionary of 46 items
2018-05-02 23:33:35,647 : MainThread : INFO : sample=0.001 downsamples 46 most-common words
2018-05-02 23:33:35,648 : MainThread : INFO : downsampling leaves estimated 14 word corpus (25.7% of prior 57)
2018-05-02 23:33:35,649 : MainThread : INFO : estimated required memory for 4